In [1]:
import time
import web3
from web3 import Web3, HTTPProvider

w3 = Web3(HTTPProvider("http://129.114.58.xx:8505"))
from web3.middleware import geth_poa_middleware

# inject the poa compatibility middleware to the innermost layer
w3.middleware_stack.inject(geth_poa_middleware, layer=0)
w3.eth.enable_unaudited_features()

# set pre-funded account as sender
w3.eth.defaultAccount = w3.eth.accounts[0]
w3.eth.accounts[0]

/home/01131/tg804093/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: enable_unaudited_features is deprecated in favor of doing nothing at all


# Explore Ethereum Chain

In [ ]:
w3.eth.getBlock(1)

# Smart Contract Creation and Compilation

In [34]:
from solc import compile_source
from web3.contract import ConciseContract

# Solidity source code
contract_source_code = '''
pragma solidity ^0.4.0;

contract KVstore {

  mapping(string=>string) store;

  function get(string key) public returns(string) {
    return store[key];
  }
  function set(string key, string value) public {
    store[key] = value;
  }
}
'''
compiled_sol = compile_source(contract_source_code) # Compiled source code
contract_interface = compiled_sol['<stdin>:KVstore']


# Deploy Contract

In [37]:
# Instantiate and deploy contract
kv = w3.eth.contract(abi=contract_interface['abi'], 
                     bytecode=contract_interface['bin'])

# Submit the transaction that deploys the contract
tx_hash = kv.constructor().transact()

# Wait for the transaction to be mined, and get the transaction receipt
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [38]:
tx_hash

HexBytes('0x1031ec2a4f90c45598608f331787a711d68bde207939e725f10f670b1f44cf36')

# Interact with Contract

In [43]:
kv = w3.eth.contract(address=tx_receipt.contractAddress,
                     abi=contract_interface['abi'])
tx_hash=kv.functions.set("hello", "asfsa").transact()

In [44]:
w3.eth.waitForTransactionReceipt(tx_hash)

AttributeDict({'blockHash': HexBytes('0xce2575b88187a6635fb43034e4675c13b90df7b8b8bcb0b2ff1ea5a93882d2fe'),
 'blockNumber': 1865,
 'contractAddress': None,
 'cumulativeGasUsed': 44360,
 'from': '0x1d87ddafac8337659d173a6b8d2f77652f89b78f',
 'gasUsed': 44360,
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'status': 1,
 'to': '0x100d752b324929165928b578fae4b7ce7af107de',
 'transactionHash': HexBytes('0x16e062e9d40c47b8955f1de533f57f402124839b33c1fee3bdd3e6da0f9ba25b'),
 'transactionIndex'

In [45]:
kv.functions.get("hello").call()

'asfsa'